In [2]:
# ============================================================
# 05_marker_genes_annotation.ipynb  (FULL SCRIPT)
# Markers (rank_genes_groups), heatmap, dotplot, marker-panel UMAPs,
# export tables, and OPTIONAL safe manual log2FC (no NaN/inf issues)
# ============================================================

import os
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

# -----------------------------
# Reproducibility / plotting
# -----------------------------
sc.settings.verbosity = 2
sc.settings.set_figure_params(dpi=140, frameon=False)
sc.settings.seed = 0

# -----------------------------
# Paths
# -----------------------------
IN_H5AD  = "../data/processed/SKCM_GSE134388_aPD1_umap.h5ad"   # output from Script 04
OUT_H5AD = "../data/processed/SKCM_GSE134388_aPD1_umap_markers.h5ad"

PLOT_DIR = "../results/markers"
os.makedirs(PLOT_DIR, exist_ok=True)

# -----------------------------
# Load AnnData
# -----------------------------
adata = sc.read_h5ad(IN_H5AD)
print("Loaded:", adata.shape)
print("obs cols (sample):", list(adata.obs.columns)[:15])
print("Has leiden?", "leiden" in adata.obs.columns)

if "leiden" not in adata.obs.columns:
    raise ValueError("leiden clustering not found. Run Script 04 first and save the .h5ad.")

Loaded: (3632, 2000)
obs cols (sample): ['UMAP_1', 'UMAP_2', 'Cluster', 'Celltype (malignancy)', 'Celltype (major-lineage)', 'Celltype (minor-lineage)', 'Patient', 'Sample', 'Tissue', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes']
Has leiden? True


In [3]:
# 1) Rank marker genes per cluster
# ============================================================
# Notes:
# - use_raw=True uses adata.raw if present (recommended)
# - method='wilcoxon' is common in Scanpy
# - If you want logfoldchanges from Scanpy, you generally need proper counts/log layers;
#   we also add an optional safe manual log2FC calculation later.
sc.tl.rank_genes_groups(
    adata,
    groupby="leiden",
    method="wilcoxon",
    use_raw=False,        
    pts=True             
)

# Save a quick "rank_genes_groups" plot (top genes per cluster)
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False, show=False)
out_png = os.path.join(PLOT_DIR, "rank_genes_groups_top20.png")
plt.savefig(out_png, bbox_inches="tight")
plt.close()
print("Saved:", out_png)

ranking genes
    finished (0:00:01)


c:\Users\mmsid\Documents\github\My Project\Ishizuka_scRNAseq_Immunotherapy\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:484: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
c:\Users\mmsid\Documents\github\My Project\Ishizuka_scRNAseq_Immunotherapy\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:484: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
c:\Users\mmsid\Documents\github\My Project\Ishizuka_scRNAseq_Immunotherapy\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:484: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
c:\Users\mmsid\Documents\github\My Project\Ishizuka_scRNAseq_Immunotherapy\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:484: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
c:\Users\mmsid\Documents\git

Saved: ../results/markers\rank_genes_groups_top20.png


In [4]:
# 2) Export full marker table (all clusters)
# ============================================================
df_all = sc.get.rank_genes_groups_df(adata, group=None)  # all clusters
out_csv = os.path.join(PLOT_DIR, "markers_rank_genes_groups_all_clusters.csv")
df_all.to_csv(out_csv, index=False)
print("Saved:", out_csv)
print(df_all.head(10))

Saved: ../results/markers\markers_rank_genes_groups_all_clusters.csv
  group    names     scores  logfoldchanges          pvals      pvals_adj  \
0     0    TAGLN  43.722889             NaN   0.000000e+00   0.000000e+00   
1     0    ACTA2  42.861031             NaN   0.000000e+00   0.000000e+00   
2     0    CALD1  42.636620             NaN   0.000000e+00   0.000000e+00   
3     0   IGFBP7  40.685081             NaN   0.000000e+00   0.000000e+00   
4     0    ADIRF  40.127903             NaN   0.000000e+00   0.000000e+00   
5     0     MYL9  39.722282             NaN   0.000000e+00   0.000000e+00   
6     0     CAV1  38.798206             NaN   0.000000e+00   0.000000e+00   
7     0  SPARCL1  38.012074             NaN   0.000000e+00   0.000000e+00   
8     0     TPM1  36.494843             NaN  1.338739e-291  2.974974e-289   
9     0     MYLK  35.749496             NaN  6.735589e-280  1.347118e-277   

   pct_nz_group  pct_nz_reference  
0           1.0               1.0  
1          

In [5]:
# 3) Build a heatmap using TOP genes per cluster (cleaner)
#    -> this shows genes on columns, clusters on rows
# ============================================================
TOP_N = 5  # change to 5/10/15 depending on readability

# Pick top N marker genes per cluster from df_all
# If df_all contains 'logfoldchanges' but it is NaN, we will sort by 'scores' (wilcoxon z-score-like)
sort_col = "scores" if "scores" in df_all.columns else "pvals_adj"

genes_by_cluster = (
    df_all.sort_values(["group", sort_col], ascending=[True, False])
         .groupby("group")["names"]
         .head(TOP_N)
         .tolist()
)

# Remove duplicates while keeping order
seen = set()
genes_top = [g for g in genes_by_cluster if not (g in seen or seen.add(g))]

print("Selected genes for heatmap:", len(genes_top))
print("Example genes:", genes_top[:20])

# Compute dendrogram so heatmap/dotplot can cluster groups
sc.tl.dendrogram(adata, groupby="leiden")

# Heatmap (publication-like)
# standard_scale="var" scales each gene across clusters so patterns pop
sc.pl.heatmap(
    adata,
    var_names=genes_top,
    groupby="leiden",
    dendrogram=True,
    use_raw=True,
    standard_scale="var",
    swap_axes=False,   # keep genes on x-axis, clusters on y-axis
    show=False
)
out_png = os.path.join(PLOT_DIR, f"markers_heatmap_top{TOP_N}_per_cluster.png")
plt.savefig(out_png, bbox_inches="tight")
plt.close()
print("Saved:", out_png)

Selected genes for heatmap: 66
Example genes: ['TAGLN', 'ACTA2', 'CALD1', 'IGFBP7', 'ADIRF', 'CD52', 'IL32', 'CD3D', 'RPS3', 'BTG1', 'TFF3', 'CCL21', 'PROX1', 'MMRN1', 'CLDN5', 'IFI27', 'PLVAP', 'TM4SF1', 'TCF4', 'RGCC']
    using 'X_pca' with n_pcs = 50


Storing dendrogram info using `.uns['dendrogram_leiden']`
Saved: ../results/markers\markers_heatmap_top5_per_cluster.png


In [6]:
# 4) Dotplot for the same genes (easier to read than heatmap)
# ============================================================
sc.pl.dotplot(
    adata,
    var_names=genes_top,
    groupby="leiden",
    dendrogram=True,
    use_raw=True,
    standard_scale="var",
    show=False
)
out_png = os.path.join(PLOT_DIR, f"markers_dotplot_top{TOP_N}_per_cluster.png")
plt.savefig(out_png, bbox_inches="tight")
plt.close()
print("Saved:", out_png)


Saved: ../results/markers\markers_dotplot_top5_per_cluster.png


In [8]:
# Quick immune/tumor marker sanity panel on UMAP
marker_panel = [
    "PTPRC",                          # CD45 immune
    "CD3D", "CD3E", "TRAC",           # T cells
    "CD8A", "NKG7", "GNLY",           # cytotoxic / NK
    "MS4A1", "CD79A",                 # B cells
    "LYZ", "LST1", "S100A8", "S100A9",# myeloid
    "DCN", "COL1A1",                  # fibroblast
    "EPCAM", "KRT8", "KRT18",         # epithelial/tumor-like
    "MKI67"                           # cycling
]

marker_panel_present = [
    g for g in marker_panel if g in adata.var_names
]

print("Marker genes present:", len(marker_panel_present), "/", len(marker_panel))

if len(marker_panel_present) > 0:
    sc.pl.umap(
        adata,
        color=marker_panel_present[:12],
        ncols=4,
        use_raw=False,   # ✅ FIXED
        show=False
    )
    plt.savefig(os.path.join(PLOT_DIR, "umap_marker_panel_first12.png"),
                bbox_inches="tight")
    plt.close()

    if len(marker_panel_present) > 12:
        sc.pl.umap(
            adata,
            color=marker_panel_present[12:],
            ncols=4,
            use_raw=False,  # ✅ FIXED
            show=False
        )
        plt.savefig(os.path.join(PLOT_DIR, "umap_marker_panel_remaining.png"),
                    bbox_inches="tight")
        plt.close()


Marker genes present: 16 / 19
